#### Parte 3: Arquitectura de una CNN

Una arquitectura típica de CNN consiste en capas alternadas de convolución y pooling, seguidas de capas completamente conectadas:




- Una capa convolucional con 32 filtros, tamaño de kernel 3x3 y activación ReLU.
- Una capa de max pooling con tamaño de pool 2x2.
- Una segunda capa convolucional con 64 filtros, tamaño de kernel 3x3 y activación ReLU.
- Una segunda capa de max pooling con tamaño de pool 2x2.
- Una tercera capa convolucional con 64 filtros, tamaño de kernel 3x3 y activación ReLU.
- Una capa completamente conectada con 64 unidades y activación ReLU.
- Una capa de salida con 10 unidades y activación softmax (para una clasificación de 10 clases).



### Explicación

1. **Capas Convolucionales**:
   - `self.conv1`: 32 filtros, tamaño de kernel 3x3, activación ReLU.
   - `self.conv2`: 64 filtros, tamaño de kernel 3x3, activación ReLU.
   - `self.conv3`: 64 filtros, tamaño de kernel 3x3, activación ReLU.

2. **Capas de Pooling**:
   - `self.pool1`: Max pooling con tamaño 2x2.
   - `self.pool2`: Max pooling con tamaño 2x2.
   - `self.pool3`: Max pooling con tamaño 2x2.

3. **Capas Completamente Conectadas**:
   - `self.fc1`: 64 unidades, activación ReLU.
   - `self.fc2`: 10 unidades (para 10 clases de salida).

4. **Aplanado**:
   - `x = x.view(-1, 64 * 4 * 4)`: Aplanar las características antes de pasar a las capas completamente conectadas.

### Uso del Modelo

Para usar este modelo, puedes definir un conjunto de datos, un optimizador y un criterio de pérdida, y luego entrenar la red con tus datos.

In [5]:


import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Capa convolucional 1: 32 filtros, kernel 3x3, activación ReLU
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Capa de max pooling 1: tamaño del pool 2x2
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 2: 64 filtros, kernel 3x3, activación ReLU
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 2: tamaño del pool 2x2
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        # Capa convolucional 3: 64 filtros, kernel 3x3, activación ReLU
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1)
        # Capa de max pooling 3: tamaño del pool 2x2
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        # Capa completamente conectada: 64 unidades, activación ReLU
        self.fc1 = nn.Linear(64 * 4 * 4, 64)
        # Capa de salida: 10 unidades (para clasificación en 10 clases), activación softmax
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        # Aplicar la primera capa convolucional seguida de ReLU y pooling
        x = self.pool1(F.relu(self.conv1(x)))
        # Aplicar la segunda capa convolucional seguida de ReLU y pooling
        x = self.pool2(F.relu(self.conv2(x)))
        # Aplicar la tercera capa convolucional seguida de ReLU y pooling
        x = self.pool3(F.relu(self.conv3(x)))
        # Aplanar las características para la capa completamente conectada
        x = x.view(-1, 64 * 4 * 4)
        # Aplicar la primera capa completamente conectada seguida de ReLU
        x = F.relu(self.fc1(x))
        # Aplicar la capa de salida
        x = self.fc2(x)
        return x

# Crear una instancia de la red
net = CNN()

# Mostrar la arquitectura del modelo
print(net)


CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)


In [6]:

import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

# Definir transformaciones para los datos de entrenamiento y prueba
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Descargar y cargar los datos de entrenamiento CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

# Descargar y cargar los datos de prueba CIFAR-10
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


# Definir criterio de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Entrenamiento de la red
for epoch in range(2):  # Cambiar a más épocas para un entrenamiento completo
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data

        optimizer.zero_grad()  # Poner a cero los gradientes del optimizador

        outputs = net(inputs)  # Hacer una pasada hacia adelante
        loss = criterion(outputs, labels)  # Calcular la pérdida
        loss.backward()  # Hacer la pasada hacia atrás (retropropagación)
        optimizer.step()  # Actualizar los pesos

        running_loss += loss.item()
        if i % 2000 == 1999:  # imprimir cada 2000 mini-lotes
            print(f'[Epoch {epoch + 1}, Mini-lote {i + 1}] pérdida: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Entrenamiento terminado')

# Guardar el modelo entrenado
PATH = './cnn.pth'
torch.save(net.state_dict(), PATH)

# Evaluación del modelo
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precisión en el conjunto de prueba: {100 * correct / total} %')

Files already downloaded and verified
Files already downloaded and verified
[Epoch 1, Mini-lote 2000] pérdida: 2.144
[Epoch 1, Mini-lote 4000] pérdida: 1.738
[Epoch 1, Mini-lote 6000] pérdida: 1.570
[Epoch 1, Mini-lote 8000] pérdida: 1.448
[Epoch 1, Mini-lote 10000] pérdida: 1.347
[Epoch 1, Mini-lote 12000] pérdida: 1.284
[Epoch 2, Mini-lote 2000] pérdida: 1.192
[Epoch 2, Mini-lote 4000] pérdida: 1.131
[Epoch 2, Mini-lote 6000] pérdida: 1.086
[Epoch 2, Mini-lote 8000] pérdida: 1.053
[Epoch 2, Mini-lote 10000] pérdida: 1.018
[Epoch 2, Mini-lote 12000] pérdida: 0.991
Entrenamiento terminado
Precisión en el conjunto de prueba: 64.43 %
